In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv2

In [2]:
from dataset import Dataset

In [3]:
path = r'E:\knns\data\cat_dog'
width = 128
height = 128

In [4]:
loader = Dataset(path=path, width=width, height=height)

In [5]:
y = loader.load_labels(12500, 12500)

In [6]:
X = loader.load_images(path=path)

In [7]:
print(len(X))
print(len(y))

25000
25000


In [8]:
def train_test_split(X:np.ndarray, y:np.ndarray, train_split:float=0.7, val_split:float=0.2, test_split:float=0.1, random_state=None):
    if float(round(train_split + val_split + test_split)) != 1.0:
        raise ValueError("train_split, val_split and test_split must add up to 1")

    if random_state is not None:
        np.random.seed(random_state)
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    train_size = int(X.shape[0] * train_split)
    val_size = int(X.shape[0] * val_split)

    train_indices = indices[:train_size]
    val_indices = indices[train_size:train_size+val_size]
    test_indices = indices[train_size+val_size:]
    
    X_train = X[train_indices]
    y_train = y[train_indices]
    X_val = X[val_indices]
    y_val = y[val_indices]
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    return X_train, X_val, X_test, y_train, y_val, y_test

In [9]:
X_train, X_val, X_test, y_train, y_val, y_test = train_test_split(X, y)

In [10]:
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"X_val: {X_val.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")
print(f"y_val: {y_val.shape}")

X_train: (17500, 128, 128)
X_test: (2500, 128, 128)
X_val: (5000, 128, 128)
y_train: (17500,)
y_test: (2500,)
y_val: (5000,)


In [11]:
num_training:int = X_train.shape[0]
num_test:int = X_test.shape[0]
num_val:int = X_val.shape[0]

In [12]:
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"X_val: {X_val.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")
print(f"y_val: {y_val.shape}")

X_train: (17500, 128, 128)
X_test: (2500, 128, 128)
X_val: (5000, 128, 128)
y_train: (17500,)
y_test: (2500,)
y_val: (5000,)


In [13]:
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
X_val = np.reshape(X_val, (X_val.shape[0], -1))

print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"X_val: {X_val.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")
print(f"y_val: {y_val.shape}")

X_train: (17500, 16384)
X_test: (2500, 16384)
X_val: (5000, 16384)
y_train: (17500,)
y_test: (2500,)
y_val: (5000,)


In [14]:
print(np.mean(np.std(X_train, axis=1)))
print(np.var(X_train))

0.9670949573506772
0.935735957206626


In [15]:
class GaussianNaiveBayes:

    __slots__ = ['_mean', '_var', '_priors', '_classes']

    def init(self):
        self._mean = None
        self._var = None
        self._priors = None
        self._classes = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)

        # Initialize mean, var, and priors
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self._priors = np.zeros(n_classes, dtype=np.float64)

        for idx, c in enumerate(self._classes):
            X_c = X[y==c]
            self._mean[idx, :] = X_c.mean(axis=0)
            self._var[idx, :] = X_c.var(axis=0)
            self._priors[idx] = X_c.shape[0] / float(n_samples)

    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        posteriors = []

        for idx, c in enumerate(self._classes):
            prior = np.log(self._priors[idx])
            class_conditional = np.sum(np.log(self._pdf(idx, x)))
            posterior = prior + class_conditional
            posteriors.append(posterior)
        
        return self._classes[np.argmax(posteriors)]

    def _pdf(self, class_idx, x):
        mean = self._mean[class_idx]
        var = self._var[class_idx]
        numerator = np.exp(- (x-mean)**2 / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator


In [16]:
gnb = GaussianNaiveBayes()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)

In [17]:
accuracy = np.mean(y_pred == y_test) 
print(f"Model accuracy: {accuracy * 100:.2f}%")

Model accuracy: 59.88%
